# Load Datasets & Quick EDA

## FD Dataset 



In [ ]:
import warnings 

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

df = df_train.append(df_test, ignore_index=True)

df_val = pd.read_csv("/content/valid.csv")

df = df.append(df_val, ignore_index=True)
df = df.drop(columns=['id'])
df.head()

,text,label
0,Nikmati cicilan 0% hingga 12 bulan untuk pemes...,neutral
1,Kue-kue yang disajikan bikin saya bernostalgia...,positive
2,Ibu pernah bekerja di grab indonesia,neutral
3,Paling suka banget makan siang di sini ayam sa...,positive
4,Pelayanan bus DAMRI sangat baik,positive


In [ ]:
df.shape

(1000, 2)

In [ ]:
df.label.value_counts()#dataset_fd

negative    383
positive    378
neutral     239
Name: label, dtype: int64

## Challange Dataset

In [ ]:
df_challange = pd.read_csv("/content/data.csv", encoding='ISO-8859-1')

In [ ]:
cols = df_challange.columns[1:]

In [ ]:
df_challange['label'] = df_challange[cols].any(axis=1).astype(int)

In [ ]:
df_challange = df_challange.drop(columns=cols)

In [ ]:
df_challange.columns = ['text','label']

In [ ]:
df_challange.label.value_counts()

1    7309
0    5860
Name: label, dtype: int64

In [ ]:
df_challange = df_challange.replace({1: 'negative', 0: 'positive'})

In [ ]:
df_challange.head()

,text,label
0,- disaat semua cowok berusaha melacak perhatia...,negative
1,RT USER: USER siapa yang telat ngasih tau elu?...,negative
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",positive
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,positive
4,USER USER Kaum cebong kapir udah keliatan dong...,negative


## IndoNLU Dataset

In [ ]:
df_indonlu_test = pd.read_csv("/content/test_preprocess.tsv", sep='\t', header= None)
df_indonlu_train = pd.read_csv("/content/train_preprocess.tsv", sep='\t', header= None)
df_indonlu_valid = pd.read_csv("/content/valid_preprocess.tsv", sep='\t', header= None)

df_indonlu = df_indonlu_train.append(df_indonlu_test, ignore_index=True)
df_indonlu = df_indonlu.append(df_indonlu_valid, ignore_index=True)
df_indonlu.columns = ['text','label']
df_indonlu.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [ ]:
df_indonlu.label.value_counts()

positive    7359
negative    4034
neutral     1367
Name: label, dtype: int64

In [ ]:
df_indonlu.shape

(12760, 2)

# Data Cleansing

In [ ]:
print(df.shape, df_challange.shape, df_indonlu.shape)

(1000, 2) (13169, 2) (12760, 2)


In [ ]:
print(df.columns, df_challange.columns, df_indonlu.columns)

Index(['text', 'label'], dtype='object') Index(['text', 'label'], dtype='object') Index(['text', 'label'], dtype='object')


In [ ]:
df_final = df.append(df_challange, ignore_index=True)
df_final = df_final.append(df_indonlu, ignore_index=True)
df_final.shape

(26929, 2)

In [ ]:
# df_final = df_final.drop(df_final[df_final['label'] == 'neutral'].index)

In [ ]:
df_final.label.value_counts()

positive    13597
negative    11726
neutral      1606
Name: label, dtype: int64

In [ ]:
df = df_final.copy()

In [ ]:
import re
import pandas as pd
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('indonesian'))

def cleansing(sent):
  string = sent.lower()
  string = re.sub(r'[^a-zA-z0-9]',' ', string)
  string = re.sub(r'[^\w]',' ', string)

  words = nltk.word_tokenize(string)
  words = [word for word in words if word not in stop_words]

  text = ' '.join(words)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
df.shape

(26929, 2)

In [ ]:
df['text'] = df.text.apply(cleansing)
df.head()

,text,label
0,nikmati cicilan 0 12 pemesanan tiket pesawat a...,neutral
1,kue kue disajikan bikin bernostalgia tipikal k...,positive
2,grab indonesia,neutral
3,suka banget makan siang ayam sambalnya enak ba...,positive
4,pelayanan bus damri,positive


In [ ]:
data_preprocessed = df.text.tolist()
type(data_preprocessed)

list

In [ ]:
len(data_preprocessed)

26929

# Features Extraction

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect = TfidfVectorizer()
# count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print("Feature Extraction Done !")

Feature Extraction Done !


In [ ]:
import pickle

pickle.dump(count_vect, open("feature.p", "wb"))

# Test Split 

In [ ]:
from sklearn.model_selection import train_test_split

classes = df.label
classes

0         neutral
1        positive
2         neutral
3        positive
4        positive
           ...   
26924    negative
26925    negative
26926    negative
26927    negative
26928    positive
Name: label, Length: 26929, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size= 0.2)

# Modeling

## Simple NN , MLP

In [ ]:
#training
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier()
model_mlp.fit(X_train, y_train)

print("Training Selesai !")

Training Selesai !


In [ ]:
pickle.dump(model_mlp, open("model.p", "wb"))

In [ ]:
#evaluations
from sklearn.metrics import classification_report


test = model_mlp.predict(X_test)

print("Testing Selesai !")

print(classification_report(y_test, test))

Testing Selesai !
              precision    recall  f1-score   support

    negative       0.80      0.80      0.80      2335
     neutral       0.75      0.57      0.64       330
    positive       0.81      0.83      0.82      2721

    accuracy                           0.80      5386
   macro avg       0.78      0.73      0.75      5386
weighted avg       0.80      0.80      0.80      5386



In [ ]:
#predict
original_text = '''
barangnya kacau

'''

text = count_vect.transform([cleansing(original_text)])

result = model_mlp.predict(text)[0]
print("Sentiment: ", result)

Sentiment:  negative


## RNN

In [ ]:
df.head()

,text,label
0,nikmati cicilan 0 12 pemesanan tiket pesawat a...,neutral
1,kue kue disajikan bikin bernostalgia tipikal k...,positive
2,grab indonesia,neutral
3,suka banget makan siang ayam sambalnya enak ba...,positive
4,pelayanan bus damri,positive


In [ ]:
df.label.value_counts()

positive    13597
negative    11726
neutral      1606
Name: label, dtype: int64

In [ ]:
neg = df.loc[df['label']=='negative'].text.tolist()
pos = df.loc[df['label']=='positive'].text.tolist()
net = df.loc[df['label']=='neutral'].text.tolist()


neg_label = df.loc[df['label']=='negative'].label.tolist()
pos_label = df.loc[df['label']=='positive'].label.tolist()
net_label = df.loc[df['label']=='neutral'].label.tolist()

total_data = pos+neg+net
labels = pos_label+neg_label + net_label

In [ ]:
import pickle 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict

max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split= ' ', lower=True)
tokenizer.fit_on_texts(total_data)
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer, handle, protocol= pickle.HIGHEST_PROTOCOL)
  print("tokenizer.pickle has created !")

X = tokenizer.texts_to_sequences(total_data)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X)

X = pad_sequences(X)
with open('x_pad_sequences.pickle','wb') as handle:
  pickle.dump(X, handle, protocol = pickle.HIGHEST_PROTOCOL)
  print('x_pad_sequences.pickle has created !')

tokenizer.pickle has created !
x_pad_sequences.pickle has created !


In [ ]:
Y = pd.get_dummies(labels)
Y = Y.values

with open('y_labels.pickle','wb') as handle:
  pickle.dump(X, handle, protocol = pickle.HIGHEST_PROTOCOL)
  print('y_labels.pickle has created !')

y_labels.pickle has created !


In [ ]:
from sklearn.model_selection import train_test_split

file = open('x_pad_sequences.pickle','rb')
X = pickle.load(file)
file.close()

file = open('y_labels.pickle','rb')
X = pickle.load(file)
file.close()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state=1)

In [ ]:
#training
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K

embed_dim = 100
units = 64

model_rnn = Sequential()
model_rnn.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model_rnn.add(SimpleRNN(units,dropout = 0.2))
model_rnn.add(Dense(3, activation='softmax'))
sgd= optimizers.Adam(lr=0.001)
model_rnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_rnn.summary())

adam = optimizers.Adam(lr=0.001)
model_rnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='main',verbose = 1)
history = model_rnn.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test), verbose=1, callbacks=[es])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 114, 100)          10000000  
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                10560     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 10,010,755
Trainable params: 10,010,755
Non-trainable params: 0
_________________________________________________________________


None
Epoch 1/10
2155/2155 [==============================] - 574s 265ms/step - loss: 0.5491 - accuracy: 0.7687 - val_loss: 0.4201 - val_accuracy: 0.8359
Epoch 2/10
1697/2155 [======================>.......] - ETA: 2:09 - loss: 0.2188 - accuracy: 0.9195

In [ ]:
model_rnn.save('model_rnn.h5')
print("Model has Creqated !")

In [ ]:
#evaluation 
from sklearn import metrics 

predictions = model_rnn.predict(X_test)
y_pred= predictions
matrix_test = metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Testing Done !")
print(matrix_test)

In [ ]:
#prediction
import re
from keras.models import load_model 
input_text = '''
produknya kacau
'''
sentiment = ['positive','negative','neutral']

text = [cleansing(input_text)]
predicted = tokenizer.texts_to_sequences(text)
guess = pad_sequences(predicted, maxlen=X.shape[1])

model = load_model('model_rnn.h5')
prediction= model.predict(guess)
polarity = np.argmax(prediction[0])

print("Text: ", text[0])
print("Sentiment: ", sentiment[polarity])

## LSTM

In [ ]:
#training
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K

embed_dim = 100
units = 64

model_lstm = Sequential()
model_lstm.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model_lstm.add(LSTM(units,dropout = 0.2))
model_lstm.add(Dense(3, activation='softmax'))
sgd= optimizers.Adam(lr=0.001)
model_lstm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_lstm.summary())

adam = optimizers.Adam(lr=0.001)
model_lstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='main',verbose = 1)
history = model_lstm.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test), verbose=1, callbacks=[es])

In [ ]:
model_lstm.save('model_lstm.h5')
print("Model has Creqated !")

In [ ]:
#evaluation
from sklearn import metrics 

predictions = model_lstm.predict(X_test)
y_pred= predictions
matrix_test = metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Testing Done !")
print(matrix_test)

In [ ]:
#prediction
import re
from keras.models import load_model 
input_text = '''
produknya kacau
'''
sentiment = ['positive','negative','neutral']

text = [cleansing(input_text)]
predicted = tokenizer.texts_to_sequences(text)
guess = pad_sequences(predicted, maxlen=X.shape[1])

model = load_model('/content/model_lstm.h5')
prediction= model.predict(guess)
polarity = np.argmax(prediction[0])

print("Text: ", text[0])
print("Sentiment: ", sentiment[polarity])

1/1 [==============================] - 1s 716ms/step
Text:  produknya kacau
Sentiment:  positive
